<a href="https://colab.research.google.com/github/kuds/reinforce-tactics/blob/main/notebooks/llm_bot_tournament.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 LLM Bot Tournament - Reinforce Tactics

Run interactive tournaments between LLM-powered bots (OpenAI GPT, Claude, Gemini) and the SimpleBot!

**Features:**
- 🎮 Single game runner with detailed turn-by-turn logging
- 🏆 Round-robin tournament system with multiple games per matchup
- 🔑 Flexible API key configuration (environment variables or Google Colab secrets)
- 📊 Comprehensive statistics: wins, losses, draws, win rates
- 🎯 Customizable model selection (GPT-4o, Claude Sonnet, Gemini Pro, etc.)
- 🗺️ Support for all map sizes (6x6 beginner to 32x32 expert)

**Supported Bots:**
- **SimpleBot**: Built-in rule-based bot (always available)
- **OpenAIBot**: Uses OpenAI GPT models (requires `OPENAI_API_KEY`)
- **ClaudeBot**: Uses Anthropic Claude models (requires `ANTHROPIC_API_KEY`)
- **GeminiBot**: Uses Google Gemini models (requires `GOOGLE_API_KEY`)

**Quick Start:**
1. Install dependencies and clone the repository
2. Configure API keys for the bots you want to use
3. Run a single game or full tournament
4. Analyze the results!

**Estimated API Costs:**
- **GPT-4o-mini**: ~$0.0001-0.0005 per game (recommended for testing)
- **Claude Haiku**: ~$0.0001-0.0003 per game (recommended for testing)
- **Gemini Flash**: Free tier available, ~$0.0001 per game
- **GPT-4o**: ~$0.005-0.02 per game (stronger play, higher cost)
- **Claude Sonnet**: ~$0.003-0.015 per game (stronger play, higher cost)

*Costs vary based on game length and map complexity. Use mini/haiku/flash models for testing!*

## 📦 Setup and Installation

In [ ]:
# Clone the Reinforce Tactics repository if not already present
import os
from pathlib import Path

if not Path('reinforce-tactics').exists():
    print("📥 Cloning Reinforce Tactics repository...")
    !git clone https://github.com/kuds/reinforce-tactics.git
    print("✅ Repository cloned!")
else:
    print("✅ Repository already cloned")

# Change to repository directory
os.chdir('reinforce-tactics')
print(f"\n📂 Current directory: {os.getcwd()}")

In [ ]:
# Install LLM bot dependencies
# These are optional - only install for the bots you plan to use
print("📦 Installing LLM dependencies...\n")

# Install OpenAI (for GPT models)
print("Installing OpenAI...")
!pip install -q openai>=1.0.0

# Install Anthropic (for Claude models)
print("Installing Anthropic...")
!pip install -q anthropic>=0.18.0

# Install Google Generative AI (for Gemini models)
print("Installing Google Generative AI...")
!pip install -q google-generativeai>=0.4.0

# Install other dependencies if needed
!pip install -q pandas numpy

print("\n✅ All LLM dependencies installed!")

In [ ]:
# Add repository to Python path
import sys
repo_path = os.getcwd()
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)
    print(f"✅ Added to Python path: {repo_path}")
else:
    print(f"✅ Already in Python path: {repo_path}")

## 🔑 API Key Configuration

You have two options for setting API keys:

### Option 1: Direct Environment Variables (Quick Setup)
Set API keys directly in the cells below. **Note:** These will be visible in the notebook.

### Option 2: Google Colab Secrets (Recommended for Colab)
1. Click the 🔑 key icon in the left sidebar
2. Add secrets with names: `OPENAI_API_KEY`, `ANTHROPIC_API_KEY`, `GOOGLE_API_KEY`
3. Toggle "Notebook access" on for each secret

The code below will check both sources and use Colab secrets if available.

In [ ]:
import os

# Try to use Google Colab secrets first
try:
    from google.colab import userdata
    print("✅ Google Colab detected - checking for secrets...\n")

    # Try to get OpenAI key from secrets
    try:
        openai_key = userdata.get('OPENAI_API_KEY')
        os.environ['OPENAI_API_KEY'] = openai_key
        print("✅ OPENAI_API_KEY loaded from Colab secrets")
    except:
        if 'OPENAI_API_KEY' not in os.environ:
            print("⚠️  OPENAI_API_KEY not found in Colab secrets")

    # Try to get Anthropic key from secrets
    try:
        anthropic_key = userdata.get('ANTHROPIC_API_KEY')
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key
        print("✅ ANTHROPIC_API_KEY loaded from Colab secrets")
    except:
        if 'ANTHROPIC_API_KEY' not in os.environ:
            print("⚠️  ANTHROPIC_API_KEY not found in Colab secrets")

    # Try to get Google key from secrets
    try:
        google_key = userdata.get('GOOGLE_API_KEY')
        os.environ['GOOGLE_API_KEY'] = google_key
        print("✅ GOOGLE_API_KEY loaded from Colab secrets")
    except:
        if 'GOOGLE_API_KEY' not in os.environ:
            print("⚠️  GOOGLE_API_KEY not found in Colab secrets")

except ImportError:
    print("ℹ️  Not running in Google Colab - using environment variables")

# Option 1: Set API keys directly (if not using Colab secrets)
# Uncomment and set your keys below if needed:

# os.environ['OPENAI_API_KEY'] = 'sk-...'
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'
# os.environ['GOOGLE_API_KEY'] = 'AI...'

print("\n" + "="*60)
print("API Key Status:")
print("="*60)
print(f"OpenAI:    {'✅ Configured' if os.environ.get('OPENAI_API_KEY') else '❌ Not set'}")
print(f"Anthropic: {'✅ Configured' if os.environ.get('ANTHROPIC_API_KEY') else '❌ Not set'}")
print(f"Google:    {'✅ Configured' if os.environ.get('GOOGLE_API_KEY') else '❌ Not set'}")
print("="*60)
print("\nℹ️  You can run tournaments with any bots that have API keys configured.")
print("   SimpleBot is always available and doesn't require an API key.")

## 📚 Import Required Modules

In [ ]:
import logging
from collections import defaultdict
from typing import Dict, List, Any, Optional, Tuple, Union

# Configure logging to see bot actions
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import game components
from reinforcetactics.core.game_state import GameState
from reinforcetactics.game.bot import SimpleBot
from reinforcetactics.utils.file_io import FileIO

# Import LLM bots (with graceful fallback)
llm_bots_available = {}

try:
    from reinforcetactics.game.llm_bot import OpenAIBot
    llm_bots_available['openai'] = OpenAIBot
    print("✅ OpenAIBot available")
except ImportError as e:
    print(f"⚠️  OpenAIBot not available: {e}")

try:
    from reinforcetactics.game.llm_bot import ClaudeBot
    llm_bots_available['claude'] = ClaudeBot
    print("✅ ClaudeBot available")
except ImportError as e:
    print(f"⚠️  ClaudeBot not available: {e}")

try:
    from reinforcetactics.game.llm_bot import GeminiBot
    llm_bots_available['gemini'] = GeminiBot
    print("✅ GeminiBot available")
except ImportError as e:
    print(f"⚠️  GeminiBot not available: {e}")

print(f"\n✅ Imports complete! {len(llm_bots_available)} LLM bot types available.")

## 🎮 Single Game Runner

Run a single game between two bots with detailed logging.

In [ ]:
def run_single_game(
    player1_bot: Union[str, type],
    player2_bot: Union[str, type],
    map_file: str = 'maps/1v1/6x6_beginner.csv',
    max_turns: int = 500,
    verbose: bool = True,
    player1_model: Optional[str] = None,
    player2_model: Optional[str] = None
) -> int:
    """
    Run a single game between two bots.

    Args:
        player1_bot: Bot class or 'simple' for SimpleBot (plays as Player 1)
        player2_bot: Bot class or 'simple' for SimpleBot (plays as Player 2)
        map_file: Path to map file (default: maps/1v1/6x6_beginner.csv)
        max_turns: Maximum number of turns to prevent infinite games (default: 500)
        verbose: Show turn-by-turn details (default: True)
        player1_model: Optional model name for player 1 LLM bot
        player2_model: Optional model name for player 2 LLM bot

    Returns:
        Winner: 1 (player 1 wins), 2 (player 2 wins), or 0 (draw)
    """
    # Load map
    map_data = FileIO.load_map(map_file)
    if map_data is None:
        raise ValueError(f"Failed to load map: {map_file}")

    # Create game state
    game_state = GameState(map_data, num_players=2)

    # Create bot instances
    def create_bot(bot_spec, player_num, model=None):
        if bot_spec == 'simple' or bot_spec is None:
            return SimpleBot(game_state, player_num)
        else:
            # It's an LLM bot class
            if model:
                return bot_spec(game_state, player_num, model=model)
            else:
                return bot_spec(game_state, player_num)

    bot1 = create_bot(player1_bot, 1, player1_model)
    bot2 = create_bot(player2_bot, 2, player2_model)
    bots = {1: bot1, 2: bot2}

    # Get bot names
    bot1_name = bot1.__class__.__name__
    bot2_name = bot2.__class__.__name__

    if verbose:
        print("\n" + "="*60)
        print(f"Game Start: {bot1_name} (P1) vs {bot2_name} (P2)")
        print(f"Map: {map_file}")
        print("="*60)

    # Play the game
    turn_count = 0
    last_gold = {1: game_state.player_gold[1], 2: game_state.player_gold[2]}

    while not game_state.game_over and turn_count < max_turns:
        current_player = game_state.current_player
        current_bot = bots[current_player]
        bot_name = bot1_name if current_player == 1 else bot2_name

        # Show turn info
        if verbose:
            print(f"\n--- Turn {turn_count + 1} - {bot_name} (P{current_player}) ---")
            print(f"  Gold: P1={game_state.player_gold[1]}, P2={game_state.player_gold[2]}")

        # Bot takes turn
        try:
            current_bot.take_turn()
        except Exception as e:
            print(f"⚠️  Error during {bot_name} turn: {e}")
            # Bot forfeits on error
            game_state.game_over = True
            game_state.winner = 1 if current_player == 2 else 2
            break

        # Show gold changes
        if verbose:
            gold_change = game_state.player_gold[current_player] - last_gold[current_player]
            if gold_change != 0:
                print(f"  Gold change: {gold_change:+d}")
            last_gold[current_player] = game_state.player_gold[current_player]

        turn_count += 1

        # Check for game over
        if game_state.game_over:
            break

    # Determine winner
    if game_state.game_over and game_state.winner:
        winner = game_state.winner
        winner_name = bot1_name if winner == 1 else bot2_name
    elif turn_count >= max_turns:
        winner = 0
        winner_name = "Draw (max turns)"
    else:
        winner = 0
        winner_name = "Draw"

    if verbose:
        print("\n" + "="*60)
        print(f"Game Over! Winner: {winner_name}")
        print(f"Total turns: {turn_count}")
        print(f"Final gold - P1: {game_state.player_gold[1]}, P2: {game_state.player_gold[2]}")
        print("="*60 + "\n")

    return winner

print("✅ run_single_game() function defined")

## 🏆 Tournament Runner

Run a round-robin tournament between multiple bots.

In [ ]:
def run_tournament(
    bots: List[Tuple[str, Union[str, type], Optional[str]]],
    map_file: str = 'maps/1v1/6x6_beginner.csv',
    games_per_matchup: int = 2,
    max_turns: int = 500
) -> Dict[str, Any]:
    """
    Run a round-robin tournament between multiple bots.

    Args:
        bots: List of (name, bot_class_or_'simple', optional_model) tuples
        map_file: Path to map file (default: maps/1v1/6x6_beginner.csv)
        games_per_matchup: Games per side (total = 2 * games_per_matchup)
        max_turns: Maximum turns per game (default: 500)

    Returns:
        Dictionary with tournament results and standings

    Example:
        bots = [
            ('SimpleBot', 'simple', None),
            ('GPT-4o-mini', OpenAIBot, 'gpt-4o-mini'),
            ('Claude', ClaudeBot, None)  # Uses default model
        ]
        results = run_tournament(bots)
    """
    if len(bots) < 2:
        raise ValueError("Need at least 2 bots for a tournament")

    print("\n" + "="*70)
    print(f"🏆 TOURNAMENT START")
    print("="*70)
    print(f"Map: {map_file}")
    print(f"Participants: {len(bots)}")
    for name, bot_type, model in bots:
        model_str = f" ({model})" if model else ""
        bot_type_str = "SimpleBot" if bot_type == 'simple' else bot_type.__name__
        print(f"  - {name}: {bot_type_str}{model_str}")
    print(f"Games per matchup: {games_per_matchup * 2} ({games_per_matchup} per side)")
    print("="*70 + "\n")

    # Initialize results tracking
    results = defaultdict(lambda: {'wins': 0, 'losses': 0, 'draws': 0})
    matchup_details = []

    # Generate all matchups (round-robin)
    matchups = []
    for i in range(len(bots)):
        for j in range(i + 1, len(bots)):
            matchups.append((i, j))

    total_games = len(matchups) * games_per_matchup * 2
    print(f"📊 Total matchups: {len(matchups)}")
    print(f"📊 Total games: {total_games}\n")

    game_num = 0

    # Run all matchups
    for matchup_idx, (i, j) in enumerate(matchups, 1):
        bot1_name, bot1_class, bot1_model = bots[i]
        bot2_name, bot2_class, bot2_model = bots[j]

        print(f"\n{'='*70}")
        print(f"Matchup {matchup_idx}/{len(matchups)}: {bot1_name} vs {bot2_name}")
        print(f"{'='*70}")

        matchup_results = {
            'bot1': bot1_name,
            'bot2': bot2_name,
            'bot1_wins': 0,
            'bot2_wins': 0,
            'draws': 0
        }

        # Play games_per_matchup with bot1 as player 1
        for game in range(games_per_matchup):
            game_num += 1
            print(f"\n  Game {game_num}/{total_games}: {bot1_name} (P1) vs {bot2_name} (P2)")

            winner = run_single_game(
                bot1_class, bot2_class,
                map_file=map_file,
                max_turns=max_turns,
                verbose=False,
                player1_model=bot1_model,
                player2_model=bot2_model
            )

            if winner == 1:
                results[bot1_name]['wins'] += 1
                results[bot2_name]['losses'] += 1
                matchup_results['bot1_wins'] += 1
                print(f"    ✅ {bot1_name} wins!")
            elif winner == 2:
                results[bot2_name]['wins'] += 1
                results[bot1_name]['losses'] += 1
                matchup_results['bot2_wins'] += 1
                print(f"    ✅ {bot2_name} wins!")
            else:
                results[bot1_name]['draws'] += 1
                results[bot2_name]['draws'] += 1
                matchup_results['draws'] += 1
                print(f"    ⚖️  Draw")

        # Play games_per_matchup with bot2 as player 1 (swap sides)
        for game in range(games_per_matchup):
            game_num += 1
            print(f"\n  Game {game_num}/{total_games}: {bot2_name} (P1) vs {bot1_name} (P2)")

            winner = run_single_game(
                bot2_class, bot1_class,
                map_file=map_file,
                max_turns=max_turns,
                verbose=False,
                player1_model=bot2_model,
                player2_model=bot1_model
            )

            if winner == 1:
                results[bot2_name]['wins'] += 1
                results[bot1_name]['losses'] += 1
                matchup_results['bot2_wins'] += 1
                print(f"    ✅ {bot2_name} wins!")
            elif winner == 2:
                results[bot1_name]['wins'] += 1
                results[bot2_name]['losses'] += 1
                matchup_results['bot1_wins'] += 1
                print(f"    ✅ {bot1_name} wins!")
            else:
                results[bot1_name]['draws'] += 1
                results[bot2_name]['draws'] += 1
                matchup_results['draws'] += 1
                print(f"    ⚖️  Draw")

        # Show matchup summary
        print(f"\n  Matchup Summary: {bot1_name} {matchup_results['bot1_wins']}-{matchup_results['bot2_wins']}-{matchup_results['draws']} {bot2_name}")
        matchup_details.append(matchup_results)

    # Calculate final standings
    standings = []
    for bot_name, stats in results.items():
        total_games = stats['wins'] + stats['losses'] + stats['draws']
        win_rate = stats['wins'] / total_games if total_games > 0 else 0.0

        standings.append({
            'name': bot_name,
            'wins': stats['wins'],
            'losses': stats['losses'],
            'draws': stats['draws'],
            'total': total_games,
            'win_rate': win_rate
        })

    # Sort by wins (descending), then win_rate
    standings.sort(key=lambda x: (x['wins'], x['win_rate']), reverse=True)

    # Display final standings
    print("\n\n" + "="*70)
    print("🏆 FINAL STANDINGS")
    print("="*70)
    print(f"{'Rank':<6}{'Bot':<25}{'Wins':<8}{'Losses':<8}{'Draws':<8}{'Win Rate':<10}")
    print("-"*70)

    for rank, standing in enumerate(standings, 1):
        medal = "🥇" if rank == 1 else ("🥈" if rank == 2 else ("🥉" if rank == 3 else "  "))
        print(f"{medal} {rank:<3}{standing['name']:<25}{standing['wins']:<8}{standing['losses']:<8}"
              f"{standing['draws']:<8}{standing['win_rate']:.3f}")

    print("="*70 + "\n")

    return {
        'standings': standings,
        'matchups': matchup_details,
        'map': map_file,
        'games_per_matchup': games_per_matchup
    }

print("✅ run_tournament() function defined")

## 🎯 Example Usage

### Example 1: Single Game - SimpleBot vs SimpleBot

Let's start with a simple game between two SimpleBots to test the system.

In [ ]:
# Run a single game: SimpleBot vs SimpleBot
winner = run_single_game(
    player1_bot='simple',
    player2_bot='simple',
    map_file='maps/1v1/6x6_beginner.csv',
    max_turns=100,
    verbose=True
)

print(f"\nWinner: Player {winner}" if winner else "\nResult: Draw")

### Example 2: Single Game - LLM Bot vs SimpleBot

Test an LLM bot against SimpleBot. Make sure you have the appropriate API key configured!

In [ ]:
# Run a single game: OpenAI Bot vs SimpleBot
# Uncomment and run if you have OpenAI API key configured

# if 'openai' in llm_bots_available:
#     winner = run_single_game(
#         player1_bot=llm_bots_available['openai'],
#         player2_bot='simple',
#         map_file='maps/1v1/6x6_beginner.csv',
#         max_turns=100,
#         verbose=True,
#         player1_model='gpt-4o-mini'  # Use mini model for lower cost
#     )
#     print(f"\nWinner: Player {winner}" if winner else "\nResult: Draw")
# else:
#     print("⚠️  OpenAIBot not available. Please install openai and configure API key.")

print("Uncomment the code above to run an LLM bot game")

### Example 3: Mini Tournament

Run a small tournament with available bots.

In [ ]:
# Define tournament participants
# Format: (display_name, bot_class_or_'simple', optional_model_name)

tournament_bots = [
    ('SimpleBot', 'simple', None),
]

# Add LLM bots if available and configured
if 'openai' in llm_bots_available and os.environ.get('OPENAI_API_KEY'):
    tournament_bots.append(('GPT-4o-mini', llm_bots_available['openai'], 'gpt-4o-mini'))

if 'claude' in llm_bots_available and os.environ.get('ANTHROPIC_API_KEY'):
    tournament_bots.append(('Claude Haiku', llm_bots_available['claude'], 'claude-3-haiku-20240307'))

if 'gemini' in llm_bots_available and os.environ.get('GOOGLE_API_KEY'):
    tournament_bots.append(('Gemini Flash', llm_bots_available['gemini'], 'gemini-1.5-flash'))

# Run tournament if we have at least 2 bots
if len(tournament_bots) >= 2:
    results = run_tournament(
        bots=tournament_bots,
        map_file='maps/1v1/6x6_beginner.csv',
        games_per_matchup=2,  # 2 games per side = 4 total per matchup
        max_turns=100
    )
    print("\n✅ Tournament complete! Results saved in 'results' variable.")
else:
    print("⚠️  Need at least 2 bots for a tournament.")
    print("   Configure API keys for LLM bots or add more SimpleBots for testing.")

## 🎨 Custom Model Configuration

You can specify different models for each LLM provider. Here are some examples:

### OpenAI Models

**Available models:**
- `gpt-4o-mini` (default) - Fastest and cheapest, good for testing
- `gpt-4o` - More capable, higher cost
- `gpt-4-turbo` - Previous generation, balanced

**Example:**
```python
# Using GPT-4o for stronger gameplay
winner = run_single_game(
    player1_bot=OpenAIBot,
    player2_bot='simple',
    player1_model='gpt-4o'
)
```

### Claude Models

**Available models:**
- `claude-3-haiku-20240307` (default) - Fastest and cheapest
- `claude-3-5-sonnet-20241022` - Most capable, balanced cost
- `claude-3-opus-20240229` - Highest capability, highest cost

**Example:**
```python
# Using Claude Sonnet for better strategic play
winner = run_single_game(
    player1_bot=ClaudeBot,
    player2_bot='simple',
    player1_model='claude-3-5-sonnet-20241022'
)
```

### Gemini Models

**Available models:**
- `gemini-1.5-flash` (default) - Fast and efficient, free tier available
- `gemini-1.5-pro` - More capable, higher cost
- `gemini-2.0-flash-exp` - Experimental, cutting edge

**Example:**
```python
# Using Gemini Pro for better performance
winner = run_single_game(
    player1_bot=GeminiBot,
    player2_bot='simple',
    player1_model='gemini-1.5-pro'
)
```

### Example: Tournament with Custom Models

Compare different models from different providers:

In [ ]:
# Advanced tournament: Compare different models
# Only run this if you have all API keys configured and don't mind the cost!

# advanced_bots = [
#     ('SimpleBot', 'simple', None),
#     ('GPT-4o-mini', OpenAIBot, 'gpt-4o-mini'),
#     ('GPT-4o', OpenAIBot, 'gpt-4o'),
#     ('Claude Haiku', ClaudeBot, 'claude-3-haiku-20240307'),
#     ('Claude Sonnet', ClaudeBot, 'claude-3-5-sonnet-20241022'),
#     ('Gemini Flash', GeminiBot, 'gemini-1.5-flash'),
#     ('Gemini Pro', GeminiBot, 'gemini-1.5-pro'),
# ]

# results = run_tournament(
#     bots=advanced_bots,
#     map_file='maps/1v1/6x6_beginner.csv',
#     games_per_matchup=2,
#     max_turns=100
# )

print("Uncomment the code above to run a full model comparison tournament")
print("⚠️  Warning: This will make many API calls and may incur costs!")

## 📖 Documentation

### 💰 API Cost Estimates

**OpenAI Pricing (approximate, as of 2024):**
- GPT-4o-mini: $0.15/1M input tokens, $0.60/1M output tokens
- GPT-4o: $2.50/1M input tokens, $10.00/1M output tokens
- Typical game: 2,000-10,000 tokens per side
- **Cost per game:** $0.0001-0.0005 (mini), $0.005-0.02 (4o)

**Anthropic Pricing:**
- Claude Haiku: $0.25/1M input tokens, $1.25/1M output tokens
- Claude Sonnet: $3.00/1M input tokens, $15.00/1M output tokens
- **Cost per game:** $0.0001-0.0003 (Haiku), $0.003-0.015 (Sonnet)

**Google Gemini Pricing:**
- Gemini Flash: Free tier available (15 RPM), $0.075/1M input, $0.30/1M output
- Gemini Pro: $1.25/1M input tokens, $5.00/1M output tokens
- **Cost per game:** ~$0 (Flash free tier), $0.0001-0.001 (Flash paid), $0.001-0.005 (Pro)

**Tournament Cost Estimates:**
- 3 bots, 2 games/matchup: 12 games total
- Using mini/haiku/flash: ~$0.001-0.01 total
- Using premium models: ~$0.05-0.20 total

**Cost Saving Tips:**
1. Use mini/haiku/flash models for development and testing
2. Use smaller maps (6x6) which need fewer tokens
3. Set lower `max_turns` to prevent long games
4. Use Gemini Flash free tier for unlimited testing

### ⚡ Performance Tips

**Model Selection for Different Purposes:**

**For Testing & Development:**
- ✅ GPT-4o-mini - Fast, cheap, decent strategy
- ✅ Claude Haiku - Very fast, cheap, good baseline
- ✅ Gemini Flash - Free tier, fast, great for testing

**For Competitive Play:**
- 🏆 GPT-4o - Strong strategic thinking
- 🏆 Claude Sonnet 3.5 - Excellent reasoning, good value
- 🏆 Gemini Pro 1.5 - Good balance of cost/performance

**For Research/Analysis:**
- 🔬 Claude Opus - Highest reasoning capability
- 🔬 GPT-4 Turbo - Consistent performance

**Game Speed:**
- Smaller maps (6x6) complete in 1-3 minutes per game
- Larger maps (32x32) can take 10-30 minutes per game
- LLM API calls add 1-5 seconds per turn
- SimpleBot is nearly instant

**Tournament Duration Estimates:**
- 2 bots, 4 games: ~5-15 minutes
- 3 bots, 12 games: ~15-45 minutes
- 4 bots, 24 games: ~30-90 minutes
- 5 bots, 40 games: ~1-2 hours

**Optimization Strategies:**
1. Start with 6x6 maps for quick iterations
2. Use `games_per_matchup=1` for initial testing
3. Set `max_turns=100` for faster games
4. Run tournaments with fewer bots initially
5. Use verbose=False in run_single_game() to reduce output

### 🔧 Troubleshooting

**Problem: "API key not provided" error**
- **Solution:** Make sure you've set the API key in the environment or Colab secrets
- Check the API Key Configuration section output
- Uncomment and set the API key directly in the configuration cell

**Problem: "openai package not installed" error**
- **Solution:** Run the installation cell again
- Or manually install: `!pip install openai>=1.0.0`

**Problem: Bot makes invalid moves or errors**
- **Solution:** This is expected occasionally with LLMs
- The game will skip invalid actions and continue
- Try using a more capable model (e.g., GPT-4o instead of mini)
- Check the logs to see what actions failed

**Problem: Games taking too long**
- **Solution:** Reduce `max_turns` parameter
- Use smaller maps (6x6 instead of 32x32)
- Faster models: Haiku, Flash, or mini

**Problem: API rate limits exceeded**
- **Solution:** Add delays between games
- Use free tier models (Gemini Flash)
- Reduce `games_per_matchup`
- Spread tournament over multiple sessions

**Problem: Out of memory error**
- **Solution:** Restart the notebook runtime
- Run fewer games at once
- Use smaller maps

**Problem: "Map file not found" error**
- **Solution:** Make sure you're in the reinforce-tactics directory
- Check the path: `!ls maps/1v1/`
- Use absolute paths if needed

**Problem: Import errors for LLM bots**
- **Solution:** Check that dependencies installed correctly
- Verify the repository was cloned successfully
- Make sure the repository is in your Python path

**Problem: Tournament results seem random**
- **Solution:** LLMs have inherent randomness
- Increase `games_per_matchup` for more stable results
- Temperature parameter affects consistency (set in llm_bot.py)
- SimpleBot is deterministic and provides a good baseline

**Problem: Cost concerns**
- **Solution:** Always use mini/haiku/flash for testing
- Monitor API usage in your provider dashboard
- Set spending limits in your API account
- Test with SimpleBot first (free)
- Use Gemini Flash free tier for unlimited testing

## 🚀 Advanced Usage

### Different Map Sizes

Test bots on different map complexities:

In [ ]:
# Test on different map sizes
maps = [
    'maps/1v1/6x6_beginner.csv',
    'maps/1v1/10x10_easy.csv',
    'maps/1v1/14x14_medium.csv',
]

# for map_file in maps:
#     print(f"\n{'='*60}")
#     print(f"Testing on: {map_file}")
#     print(f"{'='*60}")
#
#     winner = run_single_game(
#         player1_bot='simple',
#         player2_bot='simple',
#         map_file=map_file,
#         max_turns=200,
#         verbose=False
#     )
#     print(f"Winner: Player {winner}" if winner else "Result: Draw")

print("Uncomment the code above to test different map sizes")

## 🎓 Next Steps

**Experiment Ideas:**
1. Compare different models from the same provider
2. Test how map size affects bot performance
3. Analyze which bots excel at different strategies
4. Track game length and resource management
5. Create a "ladder" system with Elo ratings

**Code Customization:**
1. Modify system prompts in `llm_bot.py` for different strategies
2. Add logging to track specific metrics
3. Create visualization of tournament brackets
4. Export results to CSV for analysis
5. Build a web interface for live tournaments

**Advanced Tournaments:**
1. Swiss-system tournament format
2. Double elimination brackets
3. Time-limited games
4. Asymmetric maps
5. Team battles (coming soon)

**Contributing:**
- Found a bug? Open an issue on GitHub
- Have an improvement? Submit a pull request
- Share your tournament results!

**Resources:**
- Repository: https://github.com/kuds/reinforce-tactics
- Game Rules: See `reinforcetactics/game/llm_bot.py` SYSTEM_PROMPT
- Tournament Script: `scripts/tournament.py`

---

**Happy Gaming! 🎮**